In [9]:
# Imports
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
from torch.utils.data import DataLoader# Gives easier dataset managment and creates mini batches
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard


In [10]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [11]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
# Hyperparameters

in_channels = 1
num_classes = 10
#batch_sizes = 64
#learning_rates = 0.001
num_epochs = 5

In [13]:
# Load Data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)

In [14]:
# Initialize network
model = CNN(in_channels=in_channels, num_classes=num_classes)
model.to(device)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [15]:
batch_sizes = [1, 64, 128, 1024]
learning_rates = [0.1, 0.01, 0.001, 0.0001]

In [16]:
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        step = 0 
        writer = SummaryWriter(f"runs/MNIST/MiniBatchSize {batch_size} LR {learning_rate}")
        criterion = nn.CrossEntropyLoss()
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)

        for epoch in range(num_epochs):
            losses = []
            accuracies = []

        for batch_idx, (data, targets) in enumerate(train_loader):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # forward
            scores = model(data)
            loss = criterion(scores, targets)
            losses.append(loss.item())

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Calculate 'running' training accuracy
            _, predictions = scores.max(1)
            num_correct = (predictions == targets).sum()
            running_train_acc = float(num_correct) / float(data.shape[0])

            # Plot things to tensorboard
            writer.add_scalar("Training loss", loss, global_step=step)
            writer.add_scalar("Training Accuracy", running_train_acc, global_step=step)
            step += 1

        print(f'Mean loss this epoch was {sum(losses)/len(losses)}')


Mean loss this epoch was 2.3805272048374015
Mean loss this epoch was 2.3089076160132884
Mean loss this epoch was 2.3022852302471795
Mean loss this epoch was 2.3014613709688185
Mean loss this epoch was 2.310259590270931
Mean loss this epoch was 2.3024020472060895
Mean loss this epoch was 2.3014148943968165
Mean loss this epoch was 2.3012059255957857
Mean loss this epoch was 2.3073666446498717
Mean loss this epoch was 2.3021506576904103
Mean loss this epoch was 2.3015295483156053
Mean loss this epoch was 2.3011881232515834
Mean loss this epoch was 2.3035693209050065
Mean loss this epoch was 2.301566560389632
Mean loss this epoch was 2.301299959926282
Mean loss this epoch was 2.3012220697887873
